## 文本分类实例

### Step1 导入相关包

In [31]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments
from datasets import load_dataset

### Step2 加载数据集

In [32]:
dataset = load_dataset("csv",data_files="./ChnSentiCorp_htl_all.csv",split="train")
dataset = dataset.filter(lambda x:x["review"] is not None)
dataset

Found cached dataset csv (C:/Users/24532/.cache/huggingface/datasets/csv/default-a132788c3a8afaaf/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at C:\Users\24532\.cache\huggingface\datasets\csv\default-a132788c3a8afaaf\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-0b9d7ea5accace82.arrow


Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

### Step3 划分数据集

In [33]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

Loading cached split indices for dataset at C:\Users\24532\.cache\huggingface\datasets\csv\default-a132788c3a8afaaf\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-49734d5f0422b959.arrow and C:\Users\24532\.cache\huggingface\datasets\csv\default-a132788c3a8afaaf\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-1aefcef0c489defa.arrow


DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

### Step4 数据集预处理

In [34]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(example):
    tokenized_example = tokenizer(example["review"],max_length=128,truncation=True)
    tokenized_example["labels"] = example["label"]
    return tokenized_example
tokenized_datasets = datasets.map(process_function,batched=True,remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

### Step5 创建模型

In [35]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")
model.config


Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3

BertConfig {
  "_name_or_path": "hfl/rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

### Step6 创建评估函数

In [36]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")


In [37]:
def eval_metric(eval_predict):
    predictions,labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions,references=labels)
    f1 = f1_metric.compute(predictions=predictions,references=labels)
    acc.update(f1)
    return acc

### Step7 创建TrainingArgumentg

In [38]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_toke

### Step8 创建Trainer

In [39]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

### Step9 模型训练

In [40]:
trainer.train()

  0%|          | 0/330 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.6623, 'learning_rate': 1.9393939393939395e-05, 'epoch': 0.09}
{'loss': 0.5847, 'learning_rate': 1.8787878787878792e-05, 'epoch': 0.18}
{'loss': 0.5078, 'learning_rate': 1.8181818181818182e-05, 'epoch': 0.27}
{'loss': 0.4101, 'learning_rate': 1.7575757575757576e-05, 'epoch': 0.36}
{'loss': 0.4027, 'learning_rate': 1.6969696969696972e-05, 'epoch': 0.45}
{'loss': 0.3506, 'learning_rate': 1.6363636363636366e-05, 'epoch': 0.55}
{'loss': 0.36, 'learning_rate': 1.575757575757576e-05, 'epoch': 0.64}
{'loss': 0.3249, 'learning_rate': 1.5151515151515153e-05, 'epoch': 0.73}
{'loss': 0.3561, 'learning_rate': 1.4545454545454546e-05, 'epoch': 0.82}
{'loss': 0.2905, 'learning_rate': 1.3939393939393942e-05, 'epoch': 0.91}
{'loss': 0.3246, 'learning_rate': 1.3333333333333333e-05, 'epoch': 1.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.2715088129043579, 'eval_accuracy': 0.8854568854568855, 'eval_f1': 0.9162746942615241, 'eval_runtime': 26.9954, 'eval_samples_per_second': 28.783, 'eval_steps_per_second': 0.259, 'epoch': 1.0}
{'loss': 0.2924, 'learning_rate': 1.2727272727272728e-05, 'epoch': 1.09}
{'loss': 0.2631, 'learning_rate': 1.2121212121212122e-05, 'epoch': 1.18}
{'loss': 0.2939, 'learning_rate': 1.1515151515151517e-05, 'epoch': 1.27}
{'loss': 0.297, 'learning_rate': 1.0909090909090909e-05, 'epoch': 1.36}
{'loss': 0.2642, 'learning_rate': 1.0303030303030304e-05, 'epoch': 1.45}
{'loss': 0.2849, 'learning_rate': 9.696969696969698e-06, 'epoch': 1.55}
{'loss': 0.2557, 'learning_rate': 9.090909090909091e-06, 'epoch': 1.64}
{'loss': 0.2175, 'learning_rate': 8.484848484848486e-06, 'epoch': 1.73}
{'loss': 0.2374, 'learning_rate': 7.87878787878788e-06, 'epoch': 1.82}
{'loss': 0.2706, 'learning_rate': 7.272727272727273e-06, 'epoch': 1.91}
{'loss': 0.228, 'learning_rate': 6.666666666666667e-06, 'epoch': 2.0}

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.25133657455444336, 'eval_accuracy': 0.8918918918918919, 'eval_f1': 0.9216417910447762, 'eval_runtime': 26.8849, 'eval_samples_per_second': 28.901, 'eval_steps_per_second': 0.26, 'epoch': 2.0}
{'loss': 0.2337, 'learning_rate': 6.060606060606061e-06, 'epoch': 2.09}
{'loss': 0.2677, 'learning_rate': 5.4545454545454545e-06, 'epoch': 2.18}
{'loss': 0.2016, 'learning_rate': 4.848484848484849e-06, 'epoch': 2.27}
{'loss': 0.2519, 'learning_rate': 4.242424242424243e-06, 'epoch': 2.36}
{'loss': 0.2482, 'learning_rate': 3.6363636363636366e-06, 'epoch': 2.45}
{'loss': 0.243, 'learning_rate': 3.0303030303030305e-06, 'epoch': 2.55}
{'loss': 0.2144, 'learning_rate': 2.4242424242424244e-06, 'epoch': 2.64}
{'loss': 0.2093, 'learning_rate': 1.8181818181818183e-06, 'epoch': 2.73}
{'loss': 0.2415, 'learning_rate': 1.2121212121212122e-06, 'epoch': 2.82}
{'loss': 0.2534, 'learning_rate': 6.060606060606061e-07, 'epoch': 2.91}
{'loss': 0.204, 'learning_rate': 0.0, 'epoch': 3.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.24902212619781494, 'eval_accuracy': 0.8996138996138996, 'eval_f1': 0.9277777777777779, 'eval_runtime': 42.4557, 'eval_samples_per_second': 18.301, 'eval_steps_per_second': 0.165, 'epoch': 3.0}
{'train_runtime': 3357.1716, 'train_samples_per_second': 6.245, 'train_steps_per_second': 0.098, 'train_loss': 0.304474409421285, 'epoch': 3.0}


TrainOutput(global_step=330, training_loss=0.304474409421285, metrics={'train_runtime': 3357.1716, 'train_samples_per_second': 6.245, 'train_steps_per_second': 0.098, 'train_loss': 0.304474409421285, 'epoch': 3.0})

### Step10 模型评估

In [41]:
trainer.evaluate(tokenized_datasets["test"])

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.24902212619781494,
 'eval_accuracy': 0.8996138996138996,
 'eval_f1': 0.9277777777777779,
 'eval_runtime': 26.1347,
 'eval_samples_per_second': 29.731,
 'eval_steps_per_second': 0.268,
 'epoch': 3.0}

### Step11 模型预测

In [42]:
trainer.predict(tokenized_datasets["test"])

  0%|          | 0/7 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-2.222304  ,  1.8784401 ],
       [-2.451378  ,  2.5349576 ],
       [-2.6116128 ,  2.209733  ],
       ...,
       [-2.417967  ,  2.2749321 ],
       [-0.42196646, -0.19463423],
       [ 1.2045704 , -1.799126  ]], dtype=float32), label_ids=array([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1

In [43]:
from transformers import pipeline

id2_label = id2_label = {0:"差评！",1:"好评！"}
model.config.id2label = id2_label
pipe = pipeline("text-classification",model=model,tokenizer=tokenizer)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [44]:
sen = "我觉得不错！"
pipe(sen)

[{'label': '好评！', 'score': 0.9911637306213379}]